In [2]:
%pip install psycopg2

## **Creating connection with postgres SQL from python to create and manipulate db and tables**

In [17]:
import psycopg2

In [18]:
try:
    conn  = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
except psycopg2.Error as e:
    print("Error: Conuld not connect to postgres db")
    print(e)

In [19]:
try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get the curson to the database")
    print(e)

In [20]:
conn.set_session(autocommit=True)

In [21]:
try:
    cur.execute("CREATE DATABASE pizza_sales_db")
except psycopg2.Error as e:
    print(e)

database "pizza_sales_db" already exists



In [22]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

## **Making connection witg out db that was created in postgres SQL**

In [23]:
try:
    conn  = psycopg2.connect("host=127.0.0.1 dbname=pizza_sales_db   user=postgres password=root")
except psycopg2.Error as e:
    print("Error: Conuld not connect to postgres db")
    print(e)

try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get the curson to the database")
    print(e)

conn.set_session(autocommit=True)

## **As the connection is successfull we are creating all the tables within database**

In [24]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS pizzas(pizza_id int, pizza_type_id varchar, size int, price decimal);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [25]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS pizza_types(pizza_type varchar, name varchar, category varchar, ingredients varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [26]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS orders(order_id int, date date, time time);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [27]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS orders_details(order_details_id int,order_id int, pizza_id varchar, quantity int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

## **Importing data from csv to dataframes so that we can insert in to postgres SQL db**